# ensure SEI growth volume equals to porosity reduction: Test_V

In [4]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2
import pybamm as pb;import pandas as pd;import numpy as np;
import os, json,openpyxl,traceback,multiprocessing,scipy.optimize
import matplotlib.pyplot as plt;
import imageio,timeit,random,time, signal
from scipy.io import savemat,loadmat;
from pybamm import constants,exp;import matplotlib as mpl; 
fs=17; 
font = {'family' : 'DejaVu Sans','size'   : fs}
mpl.rc('font', **font)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
Path_NiallDMA = os.path.expanduser(
    "~/EnvPBGEM_Linux/SimSave/InputData/") # for Linux
BasicPath =  os.path.expanduser(
    "~/EnvPBGEM_Linux/SimSave/P2_R9_Dim")
import sys  
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(
    os.path.join(str_path_0,"wip/Rio_Code/Fun_P2"))
sys.path.append(str_path_1) 
from Fun_P2 import * # directly do this for HPC

# Set Experiment : Exp-2
index_exp = 2 # index for experiment set, can now go for 2,3,5
Scan_start = 5;    Scan_end = 6; 
Round_No = "Debug_Exp2_SEI_Only"  # index to identify different rounds of running 
pool_no = 8
Runshort = True # True

# specify cases

Indexs =np.arange(Scan_start-1,Scan_end)
index_list = Indexs+1
if Runshort == False:
    if index_exp == 2:
        tot_cyc = 6192; cyc_age = 516; # should be 6192 but now run shorter to be faster
    if index_exp == 3:
        tot_cyc = 6180; cyc_age = 515;
    if index_exp == 5:
        tot_cyc = 1170; cyc_age = 78;
else:
    if index_exp == 2:
        tot_cyc = 3; cyc_age = 1;
    if index_exp == 3:
        tot_cyc = 8; cyc_age = 2;
    if index_exp == 5:
        tot_cyc = 8; cyc_age = 2;

para_dict_Same = {
   "Ageing temperature":25, # 25, 40 
   "Total ageing cycles":int(tot_cyc), # 
   "Ageing cycles between RPT":int(cyc_age), # 
   "Update cycles for ageing": int(cyc_age),
   "Cycles within RPT":1,
   "RPT temperature":25,
   "Mesh list":[5,5,5,60,20],   # Simon uses 30
   "Para_Set": "OKane2023", # Li2023_Coupled
   "Model option":{
        "contact resistance": "true",
        "open-circuit potential": "current sigmoid",
        "thermal": "lumped",
        "SEI": "interstitial-diffusion limited",
        "SEI on cracks": "true",
        "SEI film resistance": "distributed", 
        "SEI porosity change": "true",
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "loss of active material": ("stress-driven","none"),
        #"lithium plating": "partially reversible",
        },
    "Positive electrode LAM constant proportional term [s-1]": 1e-16,
    "Current solvent concentration in the reservoir [mol.m-3]":4541.0,
    "Current electrolyte concentration in the reservoir [mol.m-3]":1000,
    "Ratio of Li-ion concentration change in electrolyte consider solvent consumption":1.0,
    'EC initial concentration in electrolyte [mol.m-3]':4541.0,
    'Typical EC concentration in electrolyte [mol.m-3]':4541.0, 
    # unchange now:
    'Dead lithium decay constant [s-1]':1e-12, 
    'Lithium plating kinetic rate constant [m.s-1]':1e-14, 
    'Negative electrode LAM constant proportional term [s-1]':1e-16, 
    'Negative electrode cracking rate':1e-25, 
}
Para_active=[
    {
        'Inner SEI lithium interstitial diffusivity [m2.s-1]':1e-18, 
        'Initial electrolyte excessive amount ratio':1, 
        'Outer SEI partial molar volume [m3.mol-1]':10e-5, 
        "Ratio of lithium moles to SEI moles": 2.0,
    },
    {
        'Inner SEI lithium interstitial diffusivity [m2.s-1]':1e-18, 
        'Initial electrolyte excessive amount ratio':1, 
        'Outer SEI partial molar volume [m3.mol-1]':10e-5, 
        "Ratio of lithium moles to SEI moles": 1.0,
    }
]

In [6]:
# Get all para - manual debug
Para_dict_list = []
for para_active in Para_active:
    para_sum = {**para_active,**para_dict_Same}
    Para_dict_list.append(para_sum)
print(f"Total scan case is {len(Para_dict_list)}")

Total scan case is 2


In [7]:
# set up experiment
Target  = f'/Exp{index_exp}_Round_{Round_No}/'
book_name_xlsx = f'Exp{index_exp}_Round_{Round_No}.xlsx';
V_max = 4.2;        V_min = 2.5; 
if index_exp ==2:
    discharge_time_mins = 0.15* 60 * 4.86491/5
    charge_time_mins = 0.5* 60 * 4.86491/5
    exp_AGE_text = [(
        f"Discharge at 1C for {discharge_time_mins} minutes or until {V_min}V", 
        f"Charge at 0.3C for {charge_time_mins} minutes or until {V_max}V",
        ),  ]  # *  setting on cycler is 516, rather than 514 in wiki
elif index_exp ==3:
    discharge_time_mins = 0.15* 60 * 4.86491/5
    charge_time_mins = 0.5* 60 * 4.86491/5
    exp_AGE_text = [(
        f"Discharge at 1C for {discharge_time_mins} minutes or until {V_min}V", 
        f"Charge at 0.3C until {V_max}V",
        f"Hold at {V_max} V until C/100",
        ),  ]   # *  setting on cycler is 515, rather than 514 in wiki
elif index_exp ==5:
    exp_AGE_text = [(
        f"Discharge at 1C until {V_min}V", 
        f"Charge at 0.3C until {V_max}V",
        f"Hold at {V_max} V until C/100",
        ),  ]  # *  78
else:
    print("Not yet implemented!")
step_AGE_CD =0;   step_AGE_CC =1;   step_AGE_CV =2;

In [8]:
# remaining set-up for experiment and create folders
sheet_name_xlsx = 'Results';
if not os.path.exists(BasicPath + Target):
   os.mkdir(BasicPath + Target);
if not os.path.exists(BasicPath + Target+"Mats"):
   os.mkdir(BasicPath + Target+"Mats");
if not os.path.exists(BasicPath + Target+"Plots"):
   os.mkdir(BasicPath + Target+"Plots");
if not os.path.exists(BasicPath + Target+"Excel"):
   os.mkdir(BasicPath + Target+"Excel");

Path_pack = [BasicPath,Target,book_name_xlsx,sheet_name_xlsx,]

exp_RPT_text = [ (
    # refill
    f"Hold at {V_max}V until C/100",
    "Rest for 1 hours (20 minute period)", 
    # 0.1C cycle 
    f"Discharge at 0.1C until {V_min} V (30 minute period)",  
    "Rest for 3 hours (20 minute period)",  
    f"Charge at 0.1C until {V_max} V (30 minute period)",
    f"Hold at {V_max}V until C/100",
    "Rest for 1 hours (20 minute period)",
    # 0.5C cycle 
    f"Discharge at 0.5C until {V_min} V (6 minute period)",  
    "Rest for 3 hours (20 minute period)",
    f"Charge at 0.5C until {V_max} V (6 minute period)",
    f"Hold at {V_max}V until C/100",
    "Rest for 3 hours (20 minute period)",  
    ) ] * 1
# step index for RPT
step_0p1C_CD = 2; step_0p1C_CC = 4;   step_0p1C_RE =3;    
step_0p5C_CD = 7;  

exp_text_list = [exp_AGE_text,  exp_RPT_text ];
cycle_no = -1; 
exp_index_pack = [cycle_no,step_AGE_CD,step_AGE_CC,step_AGE_CV,
   step_0p1C_CD, step_0p1C_CC,step_0p1C_RE , step_0p5C_CD ];


In [9]:
# define global index and dict for all experiment data - prepare for read!
Exp_Path = [
    "Expt 1 - Si-based Degradation/",
    "Expt 2,2 - C-based Degradation 2/",
    "Expt 3 - Cathode Degradation and Li-Plating/",
    "Expt 4 - Drive Cycle Aging (Control)/",
    "Expt 5 - Standard Cycle Aging (Control)/",]
Exp_head = [
    "Expt 1",
    "Expt 2,2",
    "Expt 3",
    "Expt 4",
    "Expt 5",]
Exp_1_Cell = ["A","B","J","D","E","F","K","L","M"];
Exp_1_Temp = {
    "A":"10","B":"10","J":"10",
    "D":"25","E":"25","F":"25",
    "K":"40","L":"40","M":"40",}
Temp_Cell_Exp_1 = {
    "10":["A","B","J"],
    "25":["D","E","F"],
    "40":["K","L","M"],}
Exp_2_Cell = ["A","B","C","D","E","F"];
Exp_2_Temp = {
    "A":"10","B":"10",
    "C":"25","D":"25",
    "E":"40","F":"40",}
Temp_Cell_Exp_2 = {
    "10":["A","B"],
    "25":["C","D"],
    "40":["E","F"],}
Exp_3_Cell = ["A","B","C","D","E","F","G","H","I"];
Exp_3_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25","F":"25",
    "G":"40","H":"40","I":"40"}
Temp_Cell_Exp_3 = {
    "10":["A","B","C"],
    "25":["D","E","F"],
    "40":["G","H","I"],}
Exp_4_Cell = ["A","B","C","D","E","F","G","H"];
Exp_4_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25",
    "F":"40","G":"40","H":"40",}
Temp_Cell_Exp_4 = {
    "10":["A","B","C"],
    "25":["D","E",],
    "40":["F","G","H"],}
Exp_5_Cell = ["A","B","C","D","E","F","G","H"];
Exp_5_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25",
    "F":"40","G":"40","H":"40",}
Temp_Cell_Exp_5 = {
    "10":["A","B","C"],
    "25":["D","E",],
    "40":["F","G","H"],}
Exp_All_Cell  = [Exp_1_Cell,Exp_2_Cell,Exp_3_Cell,Exp_4_Cell,Exp_5_Cell]
Exp_Temp_Cell = [Exp_1_Temp,Exp_2_Temp,Exp_3_Temp,Exp_4_Temp,Exp_5_Temp]
Temp_Cell_Exp_All = [Temp_Cell_Exp_1,Temp_Cell_Exp_2,Temp_Cell_Exp_3,Temp_Cell_Exp_4,Temp_Cell_Exp_5]
Mark_Cell_All = [
    {
    "A":"o","B":">","J":"v",
    "D":"o","E":">","F":"v",
    "K":"o","L":">","M":"v",},
    {
    "A":"10","B":"10",
    "C":"25","D":"25",
    "E":"40","F":"40",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">","F":"v",
    "G":"o","H":">","I":"v",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">",
    "F":"o","G":">","H":"v",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">",
    "F":"o","G":">","H":"v",}]
Color_Cell_All = [
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"J":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],"F":[0, 0, 0,0.7],
    "K":[1,0,0,0.4],"L":[1,0,0,0.4],"M":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"C":[0, 0, 0,0.7],
    "E":[1,0,0,0.4],"F":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],"F":[0, 0, 0,0.7],
    "G":[1,0,0,0.4],"H":[1,0,0,0.4],"I":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],
    "F":[1,0,0,0.4],"G":[1,0,0,0.4],"H":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],
    "F":[1,0,0,0.4],"G":[1,0,0,0.4],"H":[1,0,0,0.4],}]


In [10]:
# Load Niall's data
Temp_Cell_Exp = Temp_Cell_Exp_All[index_exp-1] 
Exp_Any_AllData = Read_Exp(
    Path_NiallDMA,Exp_All_Cell[index_exp-1],
    Exp_Path,Exp_head,Exp_Temp_Cell[index_exp-1],
    index_exp-1)


Finish reading Experiment!


In [11]:
########################  Output  ########################
keys_loc_RPT = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    "CCend Negative electrode roughness ratio",
    "CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    "CDend Negative electrode roughness ratio",
    "CDend Total SEI on cracks thickness [m]",
    #"REend Total SEI on cracks thickness [m]",
]
keys_tim_RPT = [
    # default: CD
    "CD Time [h]",
    "CD Terminal voltage [V]",
    #"RE Terminal voltage [V]",
]
keys_cyc_RPT = [   # default: CDend
    "Discharge capacity [A.h]",
    "Throughput capacity [A.h]",
    "CDend Total lithium capacity in particles [A.h]",
    "CDend Loss of capacity to lithium plating [A.h]",
    "CDend Loss of capacity to SEI [A.h]",
    "CDend Loss of capacity to SEI on cracks [A.h]",
    "CDend X-averaged total SEI on cracks thickness [m]",
    "CDend X-averaged negative electrode roughness ratio",
    "CDend Local ECM resistance [Ohm]",
    "CDsta Negative electrode stoichiometry", 
    "CDend Negative electrode stoichiometry",
    "CDsta Positive electrode stoichiometry", 
    "CDend Positive electrode stoichiometry",
    "CDend Negative electrode capacity [A.h]",
    "CDend Positive electrode capacity [A.h]",
]

keys_loc_AGE = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    "CCend Negative electrode roughness ratio",
    "CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    "CDend Negative electrode roughness ratio",
    "CDend Total SEI on cracks thickness [m]",
    "CDend Electrolyte diffusivity [m2.s-1]",
    "CDend Electrolyte conductivity [S.m-1]",
]
keys_tim_AGE = [];
keys_cyc_AGE = [];
keys_all_RPT = [keys_loc_RPT,keys_tim_RPT,keys_cyc_RPT];
keys_all_AGE = [keys_loc_AGE,keys_tim_AGE,keys_cyc_AGE];
keys_all = [keys_all_RPT,keys_all_AGE];


In [12]:
# Write para - 1st round:
Values_1 = []
head_keys = list(Para_dict_list[0].keys())
head_pre = [
    "Scan No","Exp No.","Y or N",
    "Error Tot%","Error SOH %","Error LLI %",
    "Error LAM NE %","Error LAM PE %","Error Res %",
    "Dry out",]

head_pos = [ "exp_AGE_text", "exp_RPT_text",
   "Cap Loss","LLI to LiP",
   "LLI to SEI","LLI to sei-on-cracks",
   "LAM to Neg","LAM to Pos",
   "Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final","Error"]
Values_1 .append([*head_pre,*head_keys,*head_pos])

for Scan_i,Para_dict_i in zip(index_list,Para_dict_list):
    value_list_temp = list(Para_dict_i.values())
    values_para = []
    for value_list_temp_i in value_list_temp:
        values_para.append(str(value_list_temp_i))
    # sequence: scan no, exp, pass or fail, mpe, dry-out, 
    mpe_all = ["Unknown","Unknown",
        "Unknown","Unknown", 
        "Unknown","Unknown",]
    Pass_Fail = "Unknown"
    DryOut = "Unknown"
    value_Pre = [str(Scan_i),index_exp,Pass_Fail,*mpe_all,DryOut,]
    values = [*value_Pre,*values_para,]
    Values_1.append(values)
write_excel_xlsx(
    BasicPath + Target+book_name_xlsx, 
    sheet_name_xlsx, Values_1)   


Successfully create a excel file


In [13]:
# scan:
fs = 13; dpi = 100;
midc_merge_all = [];Sol_RPT_all = [];Sol_AGE_all = [];
for index_i, Para_dict_i in zip(index_list,Para_dict_list):
    midc_merge,Sol_RPT,Sol_AGE = Run_P2_Opt_Timeout(
        index_i    ,  index_exp,  Para_dict_i,   Path_pack, fs,
        keys_all,   exp_text_list, exp_index_pack,
        Exp_Any_AllData,Temp_Cell_Exp, dpi,
        Plot_Exp=True,Timeout=False,Return_Sol=True,Check_Small_Time=True,)  
    midc_merge_all.append(midc_merge)
    Sol_RPT_all.append(Sol_RPT)
    Sol_AGE_all.append(Sol_AGE) 
""" 
if __name__ == "__main__":
    pool = multiprocessing.Pool(2)
    processes = [
    pool.apply_async(
        Run_P2_Opt_Timeout, 
        args=(
            index_i    ,  index_exp,  Para_dict_i,   Path_pack, fs,
            keys_all,   exp_text_list, exp_index_pack,
            Exp_Any_AllData,Temp_Cell_Exp, dpi,
            True,False,True,True
        ) )
        for index_i, Para_dict_i in zip(index_list,Para_dict_list)]
    Result = [p.get() for p in processes]  
for result in Result:
    midc_merge_all.append(result[0])
    Sol_RPT_all.append(result[1])
    Sol_AGE_all.append(result[2]) """

Start Now! Scan 5.
Scan 5: DryOut = On
Initial electrolyte amount is  5.0430496283076 mL
Scan 5: Spent 3.402 ms on Initialization


At t = 1290.52, repeated recoverable residual errors.


Scan 5: Finish break-in cycle within 1 minute, 12 seconds
Scan 5: Finish post-process for break-in cycle within 190.671 ms
Scan 5: Finish for No.0 ageing cycles within 6.208 s
Scan 5: Finish post-process for No.1 ageing cycles within 70.487 ms


At t = 1018.22, , mxstep steps taken before reaching tout.
At t = 329.701, , mxstep steps taken before reaching tout.


Scan 5: Finish for No.1 RPT cycles within 4 minutes, 1 second
Scan 5: Finish post-process for No.1 RPT cycles within 183.715 ms
Scan 5: Finish for No.1 ageing cycles within 7.282 s
Scan 5: Finish post-process for No.2 ageing cycles within 87.020 ms


...

At t = 1137.42 and h = 6.37178e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 353.994, , mxstep steps taken before reaching tout.


Scan 6: Finish for No.2 RPT cycles within 3 minutes, 21 seconds
Scan 6: Finish post-process for No.2 RPT cycles within 138.220 ms
Scan 6: Finish for No.2 ageing cycles within 5.373 s
Scan 6: Finish post-process for No.3 ageing cycles within 77.295 ms


At t = 1101.58 and h = 1.09976e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 347.134, , mxstep steps taken before reaching tout.


Scan 6: Finish for No.3 RPT cycles within 3 minutes, 58 seconds
Scan 6: Finish post-process for No.3 RPT cycles within 129.647 ms
Scan 6: Getting extra variables within 53.900 us
Scan 6: Finish all plots within 5.642 s
Successfully create a excel file
Scan 6: Successfully save json file!
Scan 6: Successfully save mat file!
Scan 6: Try saving within 11.038 ms
Succeed doing something in 9 minutes, 37 seconds
This is the end of No. 6  scan


' \nif __name__ == "__main__":\n    pool = multiprocessing.Pool(2)\n    processes = [\n    pool.apply_async(\n        Run_P2_Opt_Timeout, \n        args=(\n            index_i    ,  index_exp,  Para_dict_i,   Path_pack, fs,\n            keys_all,   exp_text_list, exp_index_pack,\n            Exp_Any_AllData,Temp_Cell_Exp, dpi,\n            True,False,True,True\n        ) )\n        for index_i, Para_dict_i in zip(index_list,Para_dict_list)]\n    Result = [p.get() for p in processes]  \nfor result in Result:\n    midc_merge_all.append(result[0])\n    Sol_RPT_all.append(result[1])\n    Sol_AGE_all.append(result[2]) '

# for case-1: "Ratio of lithium moles to SEI moles": 1.0

In [35]:
Para_dict_i = Para_dict_list[0]
CyclePack,Para = Para_init(Para_dict_i)
Sol = Sol_RPT_all[0][0]
temp_Int_ElelyExces_Ratio = 1.0 #  Para["Initial electrolyte excessive amount ratio"] 
ce_EC_0 = Para['EC initial concentration in electrolyte [mol.m-3]'] # used to calculate ce_EC_All
if temp_Int_ElelyExces_Ratio < 1:
    Int_ElelyExces_Ratio = -1;
    DryOut = "Off";
else:
    Int_ElelyExces_Ratio = temp_Int_ElelyExces_Ratio;
    DryOut = "On";
print(f"Scan {Scan_i}: DryOut = {DryOut}")
if DryOut == "On":  
    mdic_dry,Para = Initialize_mdic_dry(
        Para,Int_ElelyExces_Ratio)
else:
    mdic_dry ={}
#
L_p   =Para["Positive electrode thickness [m]"]
L_n   =Para["Negative electrode thickness [m]"]
L_s   =Para["Separator thickness [m]"]
L_y   =Para["Electrode width [m]"]   # Also update to change A_cc and therefore Q
L_z   =Para["Electrode height [m]"]
c_EC_r_old=Para["Current solvent concentration in the reservoir [mol.m-3]"]   # add two new parameters for paper 2
c_e_r_old =Para["Current electrolyte concentration in the reservoir [mol.m-3]"]
# Initial EC concentration in JR
c_EC_JR_old =Para["Bulk solvent concentration [mol.m-3]"]  # Be careful when multiplying with pore volume to get amount in mole. Because of electrolyte dry out, that will give more amount than real.   
# LLI due to electrode,Ratio of EC and lithium is 1:1 -> EC amount consumed is LLINegSEI[-1]
LLINegSEI = (
    Sol["Loss of lithium to SEI [mol]"].entries[-1] 
    - Sol["Loss of lithium to SEI [mol]"].entries[0] )
LLINegSEIcr = (
    Sol["Loss of lithium to SEI on cracks [mol]"].entries[-1]
    - 
    Sol["Loss of lithium to SEI on cracks [mol]"].entries[0]
    )
LLINegDeadLiP = (
    Sol["Loss of lithium to dead lithium plating [mol]"].entries[-1] 
    - Sol["Loss of lithium to dead lithium plating [mol]"].entries[0])
LLINegLiP = (
    Sol["Loss of lithium to lithium plating [mol]"].entries[-1] 
    - Sol["Loss of lithium to lithium plating [mol]"].entries[0])
cLi_Xavg  = Sol["X-averaged electrolyte concentration [mol.m-3]"].entries[-1] 
# Pore volume change with time:
PoreVolNeg_0 = Sol["X-averaged negative electrode porosity"].entries[0]*L_n*L_y*L_z;
PoreVolSep_0 = Sol["X-averaged separator porosity"].entries[0]*L_s*L_y*L_z;
PoreVolPos_0 = Sol["X-averaged positive electrode porosity"].entries[0]*L_p*L_y*L_z;
PoreVolNeg_1 = Sol["X-averaged negative electrode porosity"].entries[-1]*L_n*L_y*L_z;
PoreVolSep_1 = Sol["X-averaged separator porosity"].entries[-1]*L_s*L_y*L_z;
PoreVolPos_1 = Sol["X-averaged positive electrode porosity"].entries[-1]*L_p*L_y*L_z;
#############################################################################################################################  
##### Step-2 Determine How much electrolyte is added (from the reservoir to JR) or squeezed out (from JR to reservoir) ######
#######################   and finish electrolyte mixing     #################################################################  
Vol_Elely_Tot_old = Para["Current total electrolyte volume in whole cell [m3]"] 
Vol_Elely_JR_old  = Para["Current total electrolyte volume in jelly roll [m3]"] 
if Vol_Elely_Tot_old - Vol_Elely_JR_old < 0:
    print('Model error! Electrolyte in JR is larger than in the cell!')
Vol_Pore_tot_old  = PoreVolNeg_0 + PoreVolSep_0 + PoreVolPos_0    # pore volume at start time of the run
Vol_Pore_tot_new  = PoreVolNeg_1 + PoreVolSep_1 + PoreVolPos_1    # pore volume at end   time of the run, intrinsic variable 
Vol_Pore_decrease = Vol_Elely_JR_old  - Vol_Pore_tot_new # WHY Vol_Elely_JR_old not Vol_Pore_tot_old here? Because even the last state of the last solution (n-1) and the first state of the next solution (n) can be slightly different! 
# EC:lithium:SEI=2:1:1     for SEI=(CH2OCO2Li)2, but because of too many changes are required, change to 2:1:1 for now
# Because inner and outer SEI partial molar volume is the same, just set one for whole SEI
VmolSEI   = Para["Outer SEI partial molar volume [m3.mol-1]"] # 9.8e-5,
VmolLiP   = Para["Lithium metal partial molar volume [m3.mol-1]"] # 1.3e-05
VmolEC    = Para["EC partial molar volume [m3.mol-1]"]
#################   KEY EQUATION FOR DRY-OUT MODEL                   #################
# UPDATE 230525: assume the formation of dead lithium doesn’t consumed EC
Vol_EC_consumed  =  ( LLINegSEI + LLINegSEIcr   ) * 2 * VmolEC    # Mark: either with 2 or not, will decide how fast electrolyte being consumed!
Vol_Elely_need   = Vol_EC_consumed - Vol_Pore_decrease
Vol_SEILiP_increase = 1.0*(
    (LLINegSEI+LLINegSEIcr) * VmolSEI 
    #+ LLINegLiP * VmolLiP
    )    #  volume increase due to SEI+total LiP 
#################   KEY EQUATION FOR DRY-OUT MODEL                   #################

Test_V = (Vol_SEILiP_increase - Vol_Pore_decrease)/Vol_Pore_decrease*100  # This value should always be zero, but now not, which becomes the source of error!
Test_V2= (Vol_Pore_tot_old - Vol_Elely_JR_old) / Vol_Elely_JR_old * 100;  # Porosity errors due to first time step

# Start from here, there are serveral variables to be determined:
#   1) Vol_Elely_add, or Vol_Elely_squeezed; depends on conditions. easier for 'squeezed' condition
#   2) Vol_Elely_Tot_new, should always equals to Vol_Elely_Tot_old -  Vol_EC_consumed;
#   3) Vol_Elely_JR_new, for 'add' condition: see old code; for 'squeezed' condition, equals to pore volume in JR
#   4) Ratio_Dryout, for 'add' condition: see old code; for 'squeezed' condition, equals to Vol_Elely_Tot_new/Vol_Pore_tot_new 
#   5) Ratio_CeEC_JR and Ratio_CeLi_JR: for 'add' condition: see old code; for 'squeezed' condition, equals to 1 (unchanged)
#   6) c_e_r_new and c_EC_r_new: for 'add' condition: equals to old ones (unchanged); for 'squeezed' condition, need to carefully calculate     
#   7) Width_new: for 'add' condition: see old code; for 'squeezed' condition, equals to L_y (unchanged)
Vol_Elely_Tot_new = Vol_Elely_Tot_old - Vol_EC_consumed;

Scan 6: DryOut = On
Initial electrolyte amount is  5.0430496283076 mL


In [36]:
print(len(Sol_RPT_all))
[Total_Cycles,Cycle_bt_RPT,Update_Cycles,RPT_Cycles,
        Temper_i,Temper_RPT,mesh_list,submesh_strech,
        model_options] = CyclePack
Model_0 = pb.lithium_ion.DFN(options=model_options )
Model_0.variables.search("SEI thickness [m]")

2
Inner SEI thickness [m]
Outer SEI thickness [m]
Total SEI thickness [m]
X-averaged SEI thickness [m]
X-averaged inner SEI thickness [m]
X-averaged outer SEI thickness [m]
X-averaged total SEI thickness [m]


In [37]:
# check everything !
print(Para["Ratio of lithium moles to SEI moles"])
print(f"EC consume: {Vol_EC_consumed}")
print(f"Pore decrease: {Vol_Pore_decrease}")
print(f"SEI increase: {Vol_SEILiP_increase}")
print(
    f"SEI-porosity error: { (Vol_Pore_decrease-Vol_SEILiP_increase)/Vol_SEILiP_increase*100}%")
print(f"LLI to SEI mol: {LLINegSEI}")
print(f"LLI to SEI-on-crack mol: {LLINegSEIcr}")
print(PoreVolNeg_0-PoreVolNeg_1)
print((
    Sol["X-averaged total SEI thickness [m]"].entries[-1]
    -Sol["X-averaged total SEI thickness [m]"].entries[0])*L_n)

2.0
EC consume: 4.4206359227560005e-08
Pore decrease: 1.657655588152048e-08
SEI increase: 3.3153111765081753e-08
SEI-porosity error: -50.00000000307783%
LLI to SEI mol: 8.630293895243567e-06
LLI to SEI-on-crack mol: 0.00032290082375557394
1.6576555881521115e-08
1.0943096878963287e-14


# for case-2: "Ratio of lithium moles to SEI moles": 2.0

In [38]:
Para_dict_i = Para_dict_list[1]
CyclePack,Para = Para_init(Para_dict_i)
Sol = Sol_RPT_all[1][0]
temp_Int_ElelyExces_Ratio = 1.0 #  Para["Initial electrolyte excessive amount ratio"] 
ce_EC_0 = Para['EC initial concentration in electrolyte [mol.m-3]'] # used to calculate ce_EC_All
if temp_Int_ElelyExces_Ratio < 1:
    Int_ElelyExces_Ratio = -1;
    DryOut = "Off";
else:
    Int_ElelyExces_Ratio = temp_Int_ElelyExces_Ratio;
    DryOut = "On";
print(f"Scan {Scan_i}: DryOut = {DryOut}")
if DryOut == "On":  
    mdic_dry,Para = Initialize_mdic_dry(
        Para,Int_ElelyExces_Ratio)
else:
    mdic_dry ={}
#
L_p   =Para["Positive electrode thickness [m]"]
L_n   =Para["Negative electrode thickness [m]"]
L_s   =Para["Separator thickness [m]"]
L_y   =Para["Electrode width [m]"]   # Also update to change A_cc and therefore Q
L_z   =Para["Electrode height [m]"]
c_EC_r_old=Para["Current solvent concentration in the reservoir [mol.m-3]"]   # add two new parameters for paper 2
c_e_r_old =Para["Current electrolyte concentration in the reservoir [mol.m-3]"]
# Initial EC concentration in JR
c_EC_JR_old =Para["Bulk solvent concentration [mol.m-3]"]  # Be careful when multiplying with pore volume to get amount in mole. Because of electrolyte dry out, that will give more amount than real.   
# LLI due to electrode,Ratio of EC and lithium is 1:1 -> EC amount consumed is LLINegSEI[-1]
LLINegSEI = (
    Sol["Loss of lithium to SEI [mol]"].entries[-1] 
    - Sol["Loss of lithium to SEI [mol]"].entries[0] )
LLINegSEIcr = (
    Sol["Loss of lithium to SEI on cracks [mol]"].entries[-1]
    - 
    Sol["Loss of lithium to SEI on cracks [mol]"].entries[0]
    )
LLINegDeadLiP = (
    Sol["Loss of lithium to dead lithium plating [mol]"].entries[-1] 
    - Sol["Loss of lithium to dead lithium plating [mol]"].entries[0])
LLINegLiP = (
    Sol["Loss of lithium to lithium plating [mol]"].entries[-1] 
    - Sol["Loss of lithium to lithium plating [mol]"].entries[0])
cLi_Xavg  = Sol["X-averaged electrolyte concentration [mol.m-3]"].entries[-1] 
# Pore volume change with time:
PoreVolNeg_0 = Sol["X-averaged negative electrode porosity"].entries[0]*L_n*L_y*L_z;
PoreVolSep_0 = Sol["X-averaged separator porosity"].entries[0]*L_s*L_y*L_z;
PoreVolPos_0 = Sol["X-averaged positive electrode porosity"].entries[0]*L_p*L_y*L_z;
PoreVolNeg_1 = Sol["X-averaged negative electrode porosity"].entries[-1]*L_n*L_y*L_z;
PoreVolSep_1 = Sol["X-averaged separator porosity"].entries[-1]*L_s*L_y*L_z;
PoreVolPos_1 = Sol["X-averaged positive electrode porosity"].entries[-1]*L_p*L_y*L_z;
#############################################################################################################################  
##### Step-2 Determine How much electrolyte is added (from the reservoir to JR) or squeezed out (from JR to reservoir) ######
#######################   and finish electrolyte mixing     #################################################################  
Vol_Elely_Tot_old = Para["Current total electrolyte volume in whole cell [m3]"] 
Vol_Elely_JR_old  = Para["Current total electrolyte volume in jelly roll [m3]"] 
if Vol_Elely_Tot_old - Vol_Elely_JR_old < 0:
    print('Model error! Electrolyte in JR is larger than in the cell!')
Vol_Pore_tot_old  = PoreVolNeg_0 + PoreVolSep_0 + PoreVolPos_0    # pore volume at start time of the run
Vol_Pore_tot_new  = PoreVolNeg_1 + PoreVolSep_1 + PoreVolPos_1    # pore volume at end   time of the run, intrinsic variable 
Vol_Pore_decrease = Vol_Elely_JR_old  - Vol_Pore_tot_new # WHY Vol_Elely_JR_old not Vol_Pore_tot_old here? Because even the last state of the last solution (n-1) and the first state of the next solution (n) can be slightly different! 
# EC:lithium:SEI=2:1:1     for SEI=(CH2OCO2Li)2, but because of too many changes are required, change to 2:1:1 for now
# Because inner and outer SEI partial molar volume is the same, just set one for whole SEI
VmolSEI   = Para["Outer SEI partial molar volume [m3.mol-1]"] # 9.8e-5,
VmolLiP   = Para["Lithium metal partial molar volume [m3.mol-1]"] # 1.3e-05
VmolEC    = Para["EC partial molar volume [m3.mol-1]"]
#################   KEY EQUATION FOR DRY-OUT MODEL                   #################
# UPDATE 230525: assume the formation of dead lithium doesn’t consumed EC
Vol_EC_consumed  =  ( LLINegSEI + LLINegSEIcr   ) * 2 * VmolEC    # Mark: either with 2 or not, will decide how fast electrolyte being consumed!
Vol_Elely_need   = Vol_EC_consumed - Vol_Pore_decrease
Vol_SEILiP_increase = 1.0*(
    (LLINegSEI+LLINegSEIcr) * VmolSEI 
    #+ LLINegLiP * VmolLiP
    )    #  volume increase due to SEI+total LiP 
#################   KEY EQUATION FOR DRY-OUT MODEL                   #################

Test_V = (Vol_SEILiP_increase - Vol_Pore_decrease)/Vol_Pore_decrease*100  # This value should always be zero, but now not, which becomes the source of error!
Test_V2= (Vol_Pore_tot_old - Vol_Elely_JR_old) / Vol_Elely_JR_old * 100;  # Porosity errors due to first time step

# Start from here, there are serveral variables to be determined:
#   1) Vol_Elely_add, or Vol_Elely_squeezed; depends on conditions. easier for 'squeezed' condition
#   2) Vol_Elely_Tot_new, should always equals to Vol_Elely_Tot_old -  Vol_EC_consumed;
#   3) Vol_Elely_JR_new, for 'add' condition: see old code; for 'squeezed' condition, equals to pore volume in JR
#   4) Ratio_Dryout, for 'add' condition: see old code; for 'squeezed' condition, equals to Vol_Elely_Tot_new/Vol_Pore_tot_new 
#   5) Ratio_CeEC_JR and Ratio_CeLi_JR: for 'add' condition: see old code; for 'squeezed' condition, equals to 1 (unchanged)
#   6) c_e_r_new and c_EC_r_new: for 'add' condition: equals to old ones (unchanged); for 'squeezed' condition, need to carefully calculate     
#   7) Width_new: for 'add' condition: see old code; for 'squeezed' condition, equals to L_y (unchanged)
Vol_Elely_Tot_new = Vol_Elely_Tot_old - Vol_EC_consumed;

Scan 6: DryOut = On
Initial electrolyte amount is  5.0430496283076 mL


In [39]:
# check everything !
print(Para["Ratio of lithium moles to SEI moles"])
print(f"EC consume: {Vol_EC_consumed}")
print(f"Pore decrease: {Vol_Pore_decrease}")
print(f"SEI increase: {Vol_SEILiP_increase}")
print(
    f"SEI-porosity error: { (Vol_Pore_decrease-Vol_SEILiP_increase)/Vol_SEILiP_increase*100}%")
print(f"LLI to SEI mol: {LLINegSEI}")
print(f"LLI to SEI-on-crack mol: {LLINegSEIcr}")
print(PoreVolNeg_0-PoreVolNeg_1)
print((
    Sol["X-averaged total SEI thickness [m]"].entries[-1]
    -Sol["X-averaged total SEI thickness [m]"].entries[0])*L_n)

1.0
EC consume: 3.162060212619807e-08
Pore decrease: 2.37142658803322e-08
SEI increase: 2.371426588135449e-08
SEI-porosity error: -4.310862343838852e-09%
LLI to SEI mol: 8.618206903670579e-06
LLI to SEI-on-crack mol: 0.0002285244519098743
2.3714265880333047e-08
2.185554147160442e-14
